In [1]:
# Import python libraries
import glob

import pandas as pd

In [2]:
# Define paths and parameters
image_folder = '/N/slate/lramesh/AI_galaxy_dewarping/data/images_gz2/images/'
gz2_path = '/N/slate/lramesh/AI_galaxy_dewarping/data/gz2_hart16.csv'
mapping_path = '/N/slate/lramesh/AI_galaxy_dewarping/data/gz2_filename_mapping.csv'
train_percent = 0.7
val_percent = test_percent = round((1 - train_percent) / 2, 3)

In [3]:
# Read the dataset
gz2_df = pd.read_csv(gz2_path)
mapping_df = pd.read_csv(mapping_path)

# Drop unneccessary columns
drop_columns = []
for col in gz2_df.columns:
    if col != 'dr7objid' and '_debiased' not in col:
        drop_columns.append(col)
gz2_df.drop(columns=drop_columns, inplace=True)
mapping_df.drop(columns=['sample'], inplace=True)

# Merge the 2 columns
df = pd.merge(gz2_df, mapping_df, how='inner', left_on='dr7objid', right_on='objid')
df.drop(columns=['objid'], inplace=True)
df['asset_id'] = image_folder + df['asset_id'].astype(str) + '.jpg'

# Merge to filter out rows that dont have the image present
images_df = pd.DataFrame(glob.glob(f'{image_folder}*'), columns = ['asset_id'])
df = pd.merge(df, images_df, how='inner', on='asset_id')

# Shuffle the dataset for train-val-test split
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
df.head()

,dr7objid,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a03_star_or_artifact_debiased,t02_edgeon_a04_yes_debiased,t02_edgeon_a05_no_debiased,t03_bar_a06_bar_debiased,t03_bar_a07_no_bar_debiased,t04_spiral_a08_spiral_debiased,t04_spiral_a09_no_spiral_debiased,...,t10_arms_winding_a28_tight_debiased,t10_arms_winding_a29_medium_debiased,t10_arms_winding_a30_loose_debiased,t11_arms_number_a31_1_debiased,t11_arms_number_a32_2_debiased,t11_arms_number_a33_3_debiased,t11_arms_number_a34_4_debiased,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a37_cant_tell_debiased,asset_id
0,588017605220630612,0.927,0.082,0.000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000e+00,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,/N/slate/lramesh/AI_galaxy_dewarping/data/imag...
1,588023670243524721,0.478,0.467,0.055,0.824000,0.176000,0.333000,0.667000,0.333000,6.670000e-01,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,/N/slate/lramesh/AI_galaxy_dewarping/data/imag...
2,588017726014095502,0.000,0.998,0.002,0.007872,0.922567,0.445566,0.494925,0.999864,6.754379e-07,...,0.756947,0.306169,0.000000,0.000000,0.033181,0.000000,0.0,0.0,0.808360,/N/slate/lramesh/AI_galaxy_dewarping/data/imag...
3,587729385534652596,0.000,0.982,0.026,0.000352,0.980530,0.554080,0.440770,0.996885,9.426001e-04,...,0.173146,0.488209,0.281936,0.019791,0.861899,0.032265,0.0,0.0,0.063721,/N/slate/lramesh/AI_galaxy_dewarping/data/imag...
4,587729387151491256,0.927,0.073,0.000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,/N/slate/lramesh/AI_galaxy_dewarping/data/imag...


In [5]:
len(df)

239573

In [6]:
# Identify the number of images in train, val and test
val_num = test_num = round(len(df) * val_percent)
train_num = len(df) - (2 * val_num)

# Perform the dataset split
train_df = df[: train_num]
val_df = df[train_num: train_num + val_num]
test_df = df[train_num + val_num: ]

print(f'train: {len(train_df)}')
print(f'val: {len(val_df)}')
print(f'test: {len(test_df)}')

# Save the train-val-test dataset
train_df.to_csv('/N/slate/lramesh/AI_galaxy_dewarping/data/gz2_train.csv', index=False)
val_df.to_csv('/N/slate/lramesh/AI_galaxy_dewarping/data/gz2_val.csv', index=False)
test_df.to_csv('/N/slate/lramesh/AI_galaxy_dewarping/data/gz2_test.csv', index=False)

train: 167701
val: 35936
test: 35936
